In [8]:
from konlpy.tag import Mecab
from gensim.models import Word2Vec
import pandas as pd

In [12]:
# CSV 파일 경로 설정
csv_path = 'Paper_test.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_path, encoding='utf-8-sig')

# Mecab 설정
mecab = Mecab(r'C:\mecab\mecab-ko-dic')

tokenized_text = [mecab.morphs(text) for text in df['abstract'] + ' ' + df['keywords']]

# Skip-gram 모델 학습
skipgram_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, sg=1, min_count=1, workers=4)

#모델 save
skipgram_model.save('skipgram_model.bin')

In [27]:
# 저장된 모델 불러오기
loaded_model = Word2Vec.load('skipgram_model.bin')

def find_most_similar_doc_id_by_keywords(query_word):
    try:
        # 가장 유사한 단어와 유사도를 튜플로 반환
        similar_words = loaded_model.wv.most_similar(query_word, topn=1)

        if similar_words:
            most_similar_doc_id, similarity = similar_words[0]
            print(f"가장 유사한 논문 ID: {most_similar_doc_id} (유사도: {similarity:.4f})")

            # 가장 유사한 document ID의 논문 정보 출력
            similar_paper_info = df[df['keywords'].str.contains(query_word)]
            print("논문 정보:")
            print(similar_paper_info)
        else:
            print("유사한 논문을 찾을 수 없습니다.")

    except KeyError:
        print(f"'{query_word}'에 대한 정보가 없습니다.")

# 예시: '영국'에 대한 가장 유사한 논문 ID 찾기 (기준을 keywords로 변경)
query_word = '영국'
find_most_similar_doc_id_by_keywords(query_word)

가장 유사한 논문 ID: 미국 (유사도: 0.8405)
논문 정보:
                    doc_id                                            title  \
1908   CFKO201636062688544                        한국과 영국의 재난관리 체제에 대한 비교 분석   
4292   JAKO201211559437232                   지하철범죄 예방전략 : 서울 지하철의 성범죄를 중심으로   
4510   JAKO201224963707116               영국의 민간경비 의무적 자격증 및 인증계약자 제도에 관한 연구   
6339   JAKO201436438765155                     윤이상의 음악세계로 살펴본 통영국제음악제의 발전방향   
8295   JAKO201712256377711          영국의 재난관리체계 및 재난위험성 평가제도의 도입 및 적용에 관한 연구   
9420   JAKO201819063369999                          영국의 범죄정보 기반 경찰활동에 관한 연구   
9810   JAKO201900937325474  각 국별 수출물량지수의 비교 분석 : 한국, 캐나다, 브라질, 영국, 호주를 중심으로   
10107  JAKO201913261020675          영국과 한국의 민간조사원 비교 연구 - 실종자 조사시스템을 중심으로 -   
11029  JAKO202022560455058                    한국 대학생들의 미국영어와 영국영어의 모음 인지 비교   

           authors          journal  year  \
1908   윤종한;유시생;박성제  한국수자원학회:학술대회논문집  2016   
4292       노성훈;김학경       한국콘텐츠학회논문지  2012   
4510       이성기;김학경           시큐리티연구 